In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Thanks to Arun S. Maiya creator of the tutorial
https://nbviewer.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb

A very simple regression exercise. 0.800 rating with no tweak.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import tensorflow as tf

In [ ]:
import seaborn as sns

In [ ]:
!pip install ktrain

In [ ]:
import ktrain

Load training dataset

The dataset from previous competition is loaded. The scoring is the real challange. 

In [ ]:
#df=pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
#df=pd.read_csv("/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/train.csv")
#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/test_labels.csv

#/kaggle/input/d/julian3833/jigsaw-toxic-comment-classification-challenge/test.csv

df=pd.read_csv("/kaggle/input/toxicity-2022-augumentation/augumented_train_set.csv",index_col=0)

Load evaluation dataset

In [ ]:
df_valid=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)

Load comments to score

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)

Undersampling majority class

In [ ]:
df.head(1)

In [ ]:
df.describe()

In [ ]:
class_1,class_2 = df.toxic.value_counts()
print(class_1,class_2)

In [ ]:
class_2

In [ ]:
#df.toxic 

In [ ]:
df.predicted_score.value_counts()

In [ ]:

c2 = df[df["toxic"] >0]
c1 = df[df.toxic == 0]
print(len(c1),len(c2))
df_2 = c1.sample(class_2)
print(len(df_2))

In [ ]:
undersampled_df = pd.concat([df_2,c2],axis=0)
undersampled_df.drop_duplicates(subset ="comment_text",keep = False, inplace = True)

In [ ]:
df=undersampled_df

In [ ]:
print(len(undersampled_df))

Testing

Samplig for testing purpose
-to be commenteted for submission

In [ ]:
#df=df.sample(3000)
#df_valid=df_valid.sample(3000)
#comments_to_score=comments_to_score.sample(3000)

In [ ]:
comments_to_score.head()

In [ ]:
df.head()

Scoring experiments. 

In [ ]:
#df['score']=df['toxic']+2*df['severe_toxic']+3*df['obscene']+4*df['threat']+5*df['identity_hate']
#df['score']=2*df['toxic']+4*df['severe_toxic']+df['obscene']+4*df['threat']+df['insult']+4*df['identity_hate']+df['predicted_score']
#df['score']=df['toxic']+2*df['severe_toxic']+df['obscene']+df['threat']+df['insult']+df['identity_hate']+0.5*df['predicted_score']
#df['score']=df['toxic']+2*df['severe_toxic']+df['obscene']+df['threat']+df['insult']+df['identity_hate']
df['score']=0.32*df['toxic']+1.5*df['severe_toxic']+0.16*df['obscene']+1.5*df['threat']+0.64*df['insult']+1.5*df['identity_hate']
#df['score']=2*(df['toxic']+2*df['severe_toxic']+0.5*df['obscene']+df['threat']+0.5*df['insult']+df['identity_hate'])+df['predicted_score']

In [ ]:
#df['score']=2*df['score']

In [ ]:
#this was used for romanian language in original script. 
#df=df.replace(regex={r'ţ': 'ț', 'ş': 'ș','Ţ': 'Ț', 'Ş': 'Ș'})

In [ ]:
df.rename(columns={'comment_text':'sentences'},inplace=True)
#df.rename(columns={'sentiments_tags':'labels'},inplace=True)

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="score", data=df)

In [ ]:
df.score.value_counts()

Dataframe description

In [ ]:
df.describe()

Split in train and test. The validation subset is not used here. 
Impartire date in instruire test si validare

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25)
test, valid = train_test_split(test, test_size=0.05)

Prepare the dataset for tokenisation

In [ ]:
description_train = train.sentences.values

description_test = test.sentences.values


X_less_toxic =df_valid.less_toxic.values
X_more_toxic =df_valid.more_toxic.values
X_comments_to_score =comments_to_score.text.values

In [ ]:
print(len(description_train),len(description_test))

In [ ]:
#etichete'
#y_train = train.labels.to_numpy()
#y_valid = valid.labels.to_numpy()
#y_test = test.labels.to_numpy()
labels_train = train.score.values
labels_test = test.score.values

In [ ]:
print(len(labels_train),len(labels_test))

In [ ]:
x_train = description_train
y_train = labels_train
x_test = description_test
y_test = labels_test

In [ ]:
#x_train

In [ ]:
#y_test

**STEP 1: Preprocess the Data**

In [ ]:
from ktrain import text

**STEP 2: Create a Text Regression Model and Wrap in Learner**

In [ ]:
text.print_text_regression_models()

Load pretrained vectors for ktrain

Warning: for competition the fasttext vector must be downloaded here, since no internet acces is permited. 

In [ ]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          ngram_range=1, 
                                          maxlen=300, 
                                          max_features=50000)

# model = text.text_regression_model('bigru', train_data=trn, preproc=preproc)
# learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

In [ ]:
model = text.text_regression_model('bigru', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

**STEP 3: Estimate the LR**

In [ ]:
learner.lr_find(max_epochs=2)

In [ ]:
learner.lr_plot()

STEP 4: Train and Inspect the Model

Manual tuning

In [ ]:
learner.fit_onecycle(2e-4, 3)

In [ ]:
learner.plot('loss')

In [ ]:
learner.autofit(2e-5, 5)

In [ ]:
learner.plot('loss')

In [ ]:
learner.view_top_losses(n=3, preproc=preproc)

**STEP 5: Making Predictions**

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

Let's make a prediction for a random text in the validation set.

In [ ]:
idx = np.random.randint(len(x_test))
print('Description: %s' % (x_test[idx]))
print('Actual score: %s' % (y_test[idx]))

In [ ]:
predictor.predict(x_test[idx])

STEP 5: Making Predictions

Evaluation using validation_data.csv

Verificare interna.

In [ ]:
p1 = predictor.predict(X_less_toxic)
p2 = predictor.predict(X_more_toxic)
# Validation Accuracy
(p1 < p2).mean()

In [ ]:
print( "performance is :",(p1 < p2).mean())

In [ ]:
predicted_scores = predictor.predict(X_comments_to_score)


Testare

In [ ]:
comments_to_score.head()

In [ ]:
comments_to_score['score']=predicted_scores

In [ ]:
submission=comments_to_score[['score']]

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.head()

In [ ]:
df_valid['p1_score']=p1
df_valid['p2_score']=p2

In [ ]:
df_valid.to_csv('validationscores.csv')